In [1]:
from selenium import webdriver
from PIL import Image
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import os
import time
import sys

# 네이버 쇼핑 URL에서 카테고리에 맞는 각 href를 수집

In [34]:
%%time
verbose = 1
keyword = '맨투맨'
male_cat_id = 50000830
female_cat_id = 50000803
page = 1
male_shop_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={page}&pagingSize=40&viewType=list&sort=rel&cat_id={male_cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'
female_shop_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={page}&pagingSize=40&viewType=list&sort=rel&cat_id={female_cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'

driver_path = './geckodriver.exe'
# 환경변수에 추가
sys.path.append(driver_path)
options = webdriver.FirefoxOptions()

#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko')

browser = webdriver.Firefox(executable_path=driver_path, options=options)
browser.get(male_shop_url)
# 페이지 가지고올 수 있게 최대 15초까지 기다림 
browser.implicitly_wait(15)

Wall time: 6.2 s


In [35]:
%%time
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

Wall time: 554 ms


In [36]:
goods_list = soup.select_one('ul.goods_list').find_all('div', {'class':'info'})

In [37]:
len(goods_list)

44

In [60]:
for i in range(15):
    many_shop = goods_list[i].find('span', {'class':'price'}).find('a')
    if i%5 == 0:
        print(f'{i}...')
    if many_shop == None:
        shop_title_list.append(goods_list[i].find('a').attrs['title'])
        shop_href_list.append(goods_list[i].find('a').attrs['href'])
    else:
        if '판매처' in many_shop.text:
            shop_title_list.append(goods_list[i].find('a').attrs['title'])
            shop_detail = many_shop.attrs['href']
            #print(shop_detail)
            browser.get(shop_detail)
            browser.implicitly_wait(15)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            shop_href_list.append(soup.select_one('div.price_area').find('a').attrs['href'])
        else:
            shop_title_list.append(goods_list[i].find('a').attrs['title'])
            shop_href_list.append(goods_list[i].find('a').attrs['href'])

단체 맨투맨 아동용 성인용 쮸리 단체맨투맨
https://adcr.naver.com/adcr?x=8lXuGmodWfzIgISwZ7d4s////w==krZvHSi6MB2DhdBbPVGzJzPEYzYv6z2gUkkSkiEnRZIcFDpZO1s25CWtAB25dBmMSRABtN/fl2Ap29xvPwWbUu8ikHs1iNKjItWJRM0kV1xmovNG3d26WZIlpZp3KAlOa6d1/ilZQz9pGphgnGwVta/njXALWLaCDo5UC9Cf3U75XFPzb9w3TABYMCK8mgPpFO2K/UGer5NH3noDBvx4zvBtpxvndr6AqdqUbZK2rqtgBciyCCBIyCMYqmYkXnjQSIvJK6sLfDSzDWs3tWncmtLpvcm7f615EyqzqNU1T19KSAsbQvOfZIlJX4yoXvgApZtqNtTk8W/MzCFss8kvq9G1T/nJtH/APpw1ePrGbeyj72toxAQcGVED90aos5HgBYdQLPzkT9lLMG9oBsIKCzx3Pk/z6ZCMmNxwIfqmp+u8rP6Xe1vyKBkgEfUO75HwEb7Xdlp/G9Ig4AFaG6JsSpquK43LOOynX2gWQqxCJ0f0RA9BxzptHxpFD7ellwI8J8Cp2UnZetttQt9Pz9azkV6dcvDRiUbMHzJua8SVAN+ntiK8oBXCQzKoI/QQ+NwGLbcGbWLdeTBvIzbp9UFlaAjuNm8o8I5NKzKPZeeAtai2eEjm3n0qjGHpQfNg7YUpYWG8uz9RMlYg57HfXxJrgx8GhdwUd7p6V6lQm20IrERruXzhXBo2cbT4orm2rpYvQuN1KLEPRwImHTMzucHnKLhB+02gvoNjfcPcMv+4dbQdUZ/+LQJYabMThrK7FjIqVfDIyg6Euoe9ByAZyuzlu2AvUp/rNoljLd8oMHTYYxo2N0ndv+cEppWtbRTgo0lz4Qn5Ihck2oit2lSxzT8Xb0Anj2N/H8kLV07Re+hyGCRU7zJr5aXCBJzNZ0/vIN6xRKQ3DmDzNLb3o1TEFEXjIZ

http://cr2.shopping.naver.com/adcrNoti.nhn?x=VDrbcV2KOEVM2KLctXUP1%2F%2F%2F%2Fw%3D%3DsIwdEdInRTidir49%2FT2MbnqAKFYfkrjgv3tK6H%2Be9hw0zGGvRbI53cXMFK5xhei7zJbo0NSP6%2BUGG2ppjD93lyOiyTJeJUOv2TiA8o%2FUxxt4AJ48p1gvSf%2B49sywr%2BMbY9vhUHbiP3PsE1qIx7Xtt%2BRl6Skqa8Vk%2FNRlFdVPNkScR7W5gyJQ1nvFY%2BMtklQrrn8qD10eVt1lGhKzM%2BKR9ohJwRse4yLJ6KioKK%2FsNHIb%2FEX2JnxnuHg0Y9gw4Pky5G%2B1Yswqns5XOL7BW1357YuCwWIQErpecaB5%2BpIXqoHAZGil%2B6U717OlhtXtqAG1V7QJQmHpMXV27IknvclvA%2Fa9BpmK435PD9K1hykv2Il3D4WQ7Vf27UCXyouOtVYUo6403CrgW61qH1HQAqapI3Dqq7KMKtB9UY9ZX%2Fey1drcCw18zUc3oDgR6eLWX53rRyEBWTPe4nV5Wo44B5l%2FIZMa6Berw%2F0Lr17CX%2Fsic9XFNgyhNItaJhZw4GP0vaVHHYYuAd1oxkgVlpGf7%2FiKQpiUsAJ1s%2B23%2B9cBEEJ2rsJFSGZKhwds5yGiTGDy2XHKu4j349JoVGKvHl9ezQbk4uq0mbF1%2FpQrxHhQ24gIwl1j6gJOCKUniKRXxgU2aBZ2wdFHMOli47%2F0VU9v4dIW1BIRh6gel6%2B6JppsM4Ogv1qo%3D&nv_mid=18809629410&cat_id=50000830
--------------------------------------------------
특양면 베이직 맨투맨 기모 무지
http://cr2.shopping.naver.com/adcr.nhn?x=pKgYGcapr1h0krx8GQT8Mv%2F%2F%2

# 실질적으로 수집하는 함수 정의 부분

## firefox browser 생성

In [2]:
def browser_on():
    driver_path = './geckodriver.exe'
    sys.path.append(driver_path)
    options = webdriver.FirefoxOptions()

    #options.add_argument('--headless')
    #options.add_argument('--no-sandbox')
    # 사람처럼 보이게
    options.add_argument("disable-gpu")   # 가속 사용 x
    #options.add_argument("lang=ko_KR")    # 가짜 플러그인 탑재
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko') # user-agent

    return webdriver.Firefox(executable_path=driver_path, options=options)

## 네이버 쇼핑몰의 href 수집

In [3]:
def href_crawler(browser, male, start_page, end_page):    
    keyword = '맨투맨'
    if male:
        cat_id = 50000830
    else:
        cat_id = 50000803
    shop_title_list = []
    shop_href_list = []
    
    for p in range(start_page,end_page+1):
        print(f"{p}page crawling...")
        
        main_url = f'https://search.shopping.naver.com/search/all.nhn?origQuery={keyword}&pagingIndex={p}&pagingSize=40&viewType=list&sort=rel&cat_id={cat_id}&frm=NVSHCAT&query=%EB%A7%A8%ED%88%AC%EB%A7%A8'
        try:
            browser.get(main_url)
            # 페이지 가지고올 수 있게 최대 15초까지 기다림 
            browser.implicitly_wait(15)
            time.sleep(3)

            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            goods_list = soup.select_one('ul.goods_list').find_all('div', {'class':'info'})

            for i in range(len(goods_list)):
                many_shop = goods_list[i].find('span', {'class':'price'}).find('a')
                if i%5 == 0:
                    print(f'{i}...')
                if many_shop == None:
                    shop_title_list.append(goods_list[i].find('a').attrs['title'])
                    shop_href_list.append(goods_list[i].find('a').attrs['href'])
                else:
                    if '판매처' in many_shop.text:
                        shop_title_list.append(goods_list[i].find('a').attrs['title'])
                        shop_detail = many_shop.attrs['href']
                        #print(shop_detail)
                        browser.get(shop_detail)
                        browser.implicitly_wait(15)
                        html = browser.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        shop_href_list.append(soup.select_one('div.price_area').find('a').attrs['href'])
                    else:
                        shop_title_list.append(goods_list[i].find('a').attrs['title'])
                        shop_href_list.append(goods_list[i].find('a').attrs['href'])
        except:
            continue
    return shop_title_list, shop_href_list

In [4]:
%%time
browser = browser_on()

Wall time: 5.31 s


In [5]:
%%time
_, male_href_list = href_crawler(browser,True, 1,1)

1page crawling...
0...
5...
10...
15...
20...
25...
30...
35...
40...
Wall time: 39.7 s


In [ ]:
import pandas as pd
import numpy as np

In [32]:
def save_href_list(href, title):
    href_df = pd.DataFrame(np.array(href), columns=['href'])
    href_df['title'] = np.array(title)
    href_df.to_csv('./href_list.csv', mode='w')    

In [34]:
save_href_list(male_href_list, _)

In [35]:
dataset = pd.read_csv('./href_list.csv')

In [36]:
dataset = dataset.drop(['Unnamed: 0'], axis = 1)

In [45]:
male_href_list = dataset['href'].values

In [46]:
print(len(male_href_list))

44


In [47]:
def image_scrapping(href_list, save_path):
    over_weight = 0
    browser = browser_on()
    for idx, href in enumerate(href_list):
        print(f'{idx+1}/{len(href_list)} {save_path} screent shot...')
        try:
            if idx%500 == 0:
                browser.quit()
                browser = browser_on()
            browser.get(href)
            browser.implicitly_wait(15)
            time.sleep(7)
            # 팝업창 닫기
#             if len(browser.window_handles) > 1:
#                 for i in range(1,len(browser.window_handles)):
#                     browser.switch_to.window(browser.window_handles[i])
#                     time.sleep(0.5)
#                     browser.close()
#                 browser.switch_to.window(browser.window_handles[0])
#             time.sleep(2)

            # from here http://stackoverflow.com/questions/1145850/how-to-get-height-of-entire-document-with-javascript
            js = 'return Math.max( document.body.scrollHeight, document.body.offsetHeight,  document.documentElement.clientHeight,  document.documentElement.scrollHeight,  document.documentElement.offsetHeight);'
            scrollheight = browser.execute_script(js)
            if scrollheight > 20000:
                print(f'scrollheight over: {scrollheight} !!')
                over_weight +=1
                print(f'over_weight_count : {over_weight}')
                continue
            scale = 0.8
            browser.execute_script(f'document.body.style.MozTransform = "scale({scale})";')
            time.sleep(2)

            slices = []
            offset = 0
            while offset < scrollheight:
                browser.execute_script(f"window.scrollTo(0, {offset});")
                time.sleep(1)
                img = Image.open(BytesIO(browser.get_screenshot_as_png()))
                offset += img.size[1]
                slices.append(img)
                # print (f'{offset} / {scrollheight}')
                time.sleep(1)


            screenshot = Image.new('RGB', (slices[0].size[0], scrollheight))
            offset = 0
            for img in slices:
                screenshot.paste(img, (0, offset))
                offset += img.size[1]
            screenshot.save(f'./img/{save_path}/screen_shot_{idx}.png')
            time.sleep(3)
        except:
            print('error!')
            continue

In [48]:
image_scrapping(male_href_list, 'male_size')

1/44 male_size screent shot...
scrollheight over: 26091 !!
over_weight_count : 1
2/44 male_size screent shot...
scrollheight over: 24759 !!
over_weight_count : 2
3/44 male_size screent shot...
scrollheight over: 26628 !!
over_weight_count : 3
4/44 male_size screent shot...
5/44 male_size screent shot...
6/44 male_size screent shot...
7/44 male_size screent shot...
8/44 male_size screent shot...
scrollheight over: 208876 !!
over_weight_count : 4
9/44 male_size screent shot...
scrollheight over: 23595 !!
over_weight_count : 5
10/44 male_size screent shot...
11/44 male_size screent shot...
12/44 male_size screent shot...
13/44 male_size screent shot...
scrollheight over: 51434 !!
over_weight_count : 6
14/44 male_size screent shot...
scrollheight over: 169347 !!
over_weight_count : 7
15/44 male_size screent shot...
scrollheight over: 20201 !!
over_weight_count : 8
16/44 male_size screent shot...
scrollheight over: 51429 !!
over_weight_count : 9
17/44 male_size screent shot...
18/44 male_si